# Лекция: Компьютерное зрение
## Классические алгормимы компьютерного зрения

**Преподаватель:** Весельев Александр, Т-Банк

**Аудитория:** ФКН ВШЭ, МФТИ

---

## Feature Detectors and Descriptors

### Harris Corner Detector

Способ найти на картинке *углы*: места, где изображение меняется сразу в двух направлениях (и по X, и по Y).

*Идея*: при небольшом сдвиге окна (патча) вокруг точки:

- в угле картинка плохо совпадёт при любом маленьком сдвиге
- на ребре – совпадёт при сдвиге вдоль ребра
- на ровном – совпадёт почти всегда.

![](./assets/lesson_2_image_1.png)

#### Функция энергии

Пусть изображение – функция яркости:
$$I(x,y)$$

Берём маленькое окно $W$ вокруг точки $(x,y)$ и смотрим, насколько изменится картинка, если сдвинуть окно на $(u,v)$:
$$
E(u,v)=\sum_{(i,j)\in W} w(i,j),\bigl(I(i+u,,j+v)-I(i,,j)\bigr)^2
$$
где $w(i,j)$ — веса окна (gaussian), чтобы центр был важнее.

#### Линеаризация

Для малых $(u,v)$:
$$
I(i+u, j+v)\approx I(i,j)+I_x(i,j),u + I_y(i,j),v
$$
где
$$
I_x=\frac{\partial I}{\partial x},\quad I_y=\frac{\partial I}{\partial y}
$$

Тогда разность:
$$
I(i+u,j+v)-I(i,j)\approx I_x(i,j),u + I_y(i,j),v
$$

Подставляем в $E(u,v)$:
$$
E(u,v)\approx \sum_{(i,j)\in W} w(i,j),(I_x u + I_y v)^2
$$

$$
(I_x u + I_y v)^2 = I_x^2 u^2 + 2 I_x I_y u v + I_y^2 v^2
$$

Тогда:
$$
E(u,v)\approx u^2\sum w I_x^2+2uv\sum w I_x I_y+v^2\sum w I_y^2
$$

В матричном виде:
$$
E(u,v)\approx
\begin{bmatrix}u & v\end{bmatrix}\mathbf{M}\begin{bmatrix}u \\ v\end{bmatrix}
$$

где $\mathbf{M}$ — **матрица вторых моментов**:
$$
\mathbf{M}=\sum_{(i,j)\in W} w(i,j)\begin{bmatrix}I_x^2 & I_x I_y \\ I_x I_y & I_y^2 \end{bmatrix} = \begin{bmatrix}A & C \\ C & B\end{bmatrix}
$$
с обозначениями:
$$
A=\sum w I_x^2,\quad
B=\sum w I_y^2,\quad
C=\sum w I_x I_y
$$

#### Собственные значения и геометрический смысл

Пусть $\lambda_1,\lambda_2$ — собственные значения $\mathbf{M}$ (они $\ge 0$)

- **Ровная область**: градиенты малы $A,B,C\approx 0 \rightarrow \lambda_1\approx 0, \lambda_2\approx 0$
- **Ребро**: сильное изменение в одном направлении – одно собственное значение большое, второе маленькое $\lambda_1\gg \lambda_2\approx 0$
- **Угол**: изменения в двух направлениях – $\lambda_1 > 0, \lambda_2 > 0$

#### Скалярный отклик

$$
R = \det(\mathbf{M}) - k,(\operatorname{trace}(\mathbf{M}))^2
$$
где
$$
\det(\mathbf{M}) = \lambda_1\lambda_2,\quad
\operatorname{trace}(\mathbf{M}) = \lambda_1+\lambda_2
$$
и обычно $k\in[0.04, 0.06]$

Если $\mathbf{M}=\begin{bmatrix}A & C \\ C & B\end{bmatrix}$, то:

$$
\det(\mathbf{M}) = AB - C^2
$$
$$
\operatorname{trace}(\mathbf{M}) = A + B
$$

Значит:
$$
R = (AB - C^2) - k(A+B)^2
$$

Интерпретация:

- **Ровная область**: $R\approx 0$
- **Ребро**: одно значение большое, другое маленькое $\rightarrow \det \approx 0, \operatorname{trace} > 0 \rightarrow R$ часто **отрицательный**
- **Угол**: $A$ и $B$ большие, $\det > 0 \rightarrow R$ обычно **сильно положительный**


#### Практический алгоритм

1. Перевести в `grayscale`
2. Посчитать градиенты $I_x, I_y$ (обычно Sobel)
3. Посчитать $I_x^2, I_y^2, I_x I_y$
4. Сгладить их гауссовым фильтром
5. Для каждой точки вычислить $R$
6. Взять точки, где $R$ больше порога

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
import warnings
import glob

from pathlib import Path

def show_image(image, cmap=None):
    plt.imshow(image, cmap=cmap)
    plt.axis("off")
    plt.show()


SUDOKU_IMAGE_PATH = "assets/lesson_2_example_image_1.jpg"
OUTDOOR_FIRST_PART_PATH = "assets/lesson_2_example_image_2.jpg"
OUTDOOR_SECOND_PART_PATH = "assets/lesson_2_example_image_3.jpg"
OUTDOOR_THIRD_PART_PATH = "assets/lesson_2_example_image_4.jpg"

In [ ]:
def harris_example(img, nms: bool = False) -> None:
    title = f"Harris Corner Detection (nms is {'on' if nms else 'off'})"

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_f = np.float32(gray)

    block_size = 2
    ksize = 3
    k = 0.04

    R = cv2.cornerHarris(gray_f, blockSize=block_size, ksize=ksize, k=k)
    if not nms:
        R = cv2.dilate(R, None)
    thr = 0.01 * R.max()
    mask_thr = (R > thr)

    if nms:
        nms_ksize = 7
        kernel = np.ones((nms_ksize, nms_ksize), np.uint8)
        R_dilated = cv2.dilate(R, kernel)
        mask_localmax = (R == R_dilated)

        corners_mask = mask_thr & mask_localmax

        ys, xs = np.where(corners_mask)

        result = img.copy()
        for x, y in zip(xs, ys):
            cv2.circle(result, (int(x), int(y)), 3, (0, 0, 255), 1, cv2.LINE_AA)
    else:

        result = img.copy()
        result[mask_thr] = (0, 0, 255)

    plt.title(title)
    show_image(result[..., ::-1])

img = cv2.imread(SUDOKU_IMAGE_PATH)
harris_example(img)

In [ ]:
# harris_example(cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE))

---

*Напоминание: оператор Собеля*

In [ ]:
ori = cv2.cvtColor(cv2.imread(SUDOKU_IMAGE_PATH), cv2.COLOR_BGR2GRAY)

gradientx = cv2.Sobel(ori, ddepth=cv2.CV_32F, dx=1, dy=0)
gradienty = cv2.Sobel(ori, ddepth=cv2.CV_32F, dx=0, dy=1)
gradient = np.concat((gradientx, gradienty), axis=1)

show_image(np.abs(gradient), cmap='gray')

#### Laplacian of Gaussian (LoG)

In [ ]:
test_image = cv2.imread(OUTDOOR_FIRST_PART_PATH)
gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

blur = cv2.GaussianBlur(gray, (0, 0), sigmaX=1.6, sigmaY=1.5)

# # Эквивалентно
# logx = cv2.Sobel(blur, ddepth=cv2.CV_32F, dx=2, dy=0)
# logy = cv2.Sobel(blur, ddepth=cv2.CV_32F, dx=0, dy=2)
# log = logx + logy
log = cv2.Laplacian(blur, ddepth=cv2.CV_32F, ksize=3)

# Для визуализации
log_abs = np.abs(log)
log_vis = cv2.normalize(log_abs, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

plt.title("LoG")
show_image(log_vis, cmap="gray")

#### Difference of Gaussian (DoG)

"Дешевое" приближение `LoG`

In [ ]:
test_image = cv2.imread(OUTDOOR_FIRST_PART_PATH)
gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

sigma1 = 1.6
sigma2 = 1.6 * (2 ** 0.5)

# ksize ~ 6*sigma + 1
g1 = cv2.GaussianBlur(gray, ksize=(0, 0), sigmaX=sigma1, sigmaY=sigma1)
g2 = cv2.GaussianBlur(gray, ksize=(0, 0), sigmaX=sigma2, sigmaY=sigma2) 

dog = g1.astype(np.float32) - g2.astype(np.float32)

# Для визуализации
dog_vis = cv2.normalize(dog, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

plt.title("DoG")
show_image(dog_vis, cmap="gray")


#### Gaussian Pyramid

In [ ]:
img = test_image.copy()
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

num_levels = 5
pyramid = [img]

current = img
for _ in range(num_levels - 1):
    # (5, 5) - размер ядра для сглаживания
    current = cv2.pyrDown(current)
    pyramid.append(current)


# отрисовка пирамиды
h0, w0 = pyramid[0].shape[:2]

canvas_width = w0 + pyramid[1].shape[1]
canvas_height = h0

canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255

canvas[0:h0, 0:w0] = pyramid[0]

y_offset = 0
for level in pyramid[1:]:
    h, w = level.shape[:2]
    canvas[y_offset:y_offset+h, w0:w0+w] = level
    y_offset += h

plt.figure(figsize=(10, 6))
plt.imshow(canvas)
plt.title("Gaussian Pyramid (real relative sizes)")
plt.axis("off")
plt.show()


### SIFT

*Scale-Invariant Feature Transform*

Алгоритм для поиска признаков на изображениях, устойчивый к масштабированию и поворотам

![](./assets/lesson_2_image_2.jpg)

Шаги:
**1. Построение масштабного пространства (Scale-space construction)**

- Исходное изображение $I(x,y)$ сглаживается:

$$
L(x,y,\sigma) = G(x,y,\sigma) * I(x,y)
$$

- Строится несколько уровней размытия (scale levels) для каждого масштаба
- Масштабы группируются в **октавы**:
  - при переходе к следующей октаве изображение уменьшается в 2 раза
  - процесс повторяется

**2. Вычисление Difference-of-Gaussians (DoG)**


- Для соседних масштабов вычисляется разность:
$$
D(x,y,\sigma) = L(x,y,k\sigma) - L(x,y,\sigma)
$$
- Получается пирамида DoG-изображений

![](./assets/lesson_2_image_3.jpg)

**3. Поиск экстремумов в scale-space**

Цель — найти потенциальные ключевые точки

- Каждая точка DoG сравнивается:
  - с 8 соседями в том же масштабе
  - с 9 соседями в масштабе ниже
  - с 9 соседями в масштабе выше

- Если точка — локальный максимум или минимум, то кандидат в ключевую точку

![](./assets/lesson_2_image_4.jpg)

**4. Точная локализация ключевых точек**

Фильтрация точек

- Раскладывается ряд Тейлора для $D(x,y,\sigma)$ и откидываются точки низкой интенсивности   (`contrastThreshold`)
- Удаляются граничные точки, по аналогии с Harris detector (`edgeThreshold`)

**5. Назначение ориентации (Orientation assignment)**

Необходимо для rotation invariance

- В окрестности ключевой точки вычисляются:
  - величина градиента
  - направление градиента
- Строится гистограмма направлений
- Основной пик гистограммы задаёт ориентацию точки

**6. Построение дескриптора ключевой точки**

- Окрестность точки:
  - масштабируется
  - поворачивается по найденной ориентации

- Область делится на сетку $4 \times 4$
- В каждой ячейке строится гистограмма направлений градиента (8 направлений)
- Формируется вектор размерности:
$$
4 \times 4 \times 8 = 128
$$

**7. Нормализация дескриптора**

Цель — устойчивость к изменению освещения

- Дескриптор нормируется по L2-норме
- Значения ограничиваются

In [ ]:
img = cv2.imread(OUTDOOR_FIRST_PART_PATH)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
sift = cv2.SIFT_create()
kp = sift.detect(gray,None)
 
# img = cv2.drawKeypoints(gray, kp, img)
# # Отрисовка с направлениями и магнитудой
img = cv2.drawKeypoints(gray, kp, img, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

show_image(img)

---

#### FAST

![](./assets/lesson_2_image_5.jpg)

[link](https://habr.com/ru/articles/414459/)

FAST был одним из первых эвристических методов поиска особых точек, который завоевал большую популярность из-за своей вычислительной эффективности. Для принятия решения о том, считать заданную точку С особой или нет, в этом методе рассматривается яркость пикселей на окружности с центром в точке С и радиусом 3:
Сравнивая яркости пикселей окружности с яркостью центра C, получаем для каждого три возможных исхода (светлее, темнее, похоже):

$\begin{array}{l} {I_p} > {I_C} + t\\ {I_p} < {I_C}-t\\ {I_C}-t < {I_p} < {I_C} + t\end{array}$

Здесь I – яркость пикселей, t – некоторый заранее фиксированный порог по яркости.
Точка помечается как особая, если на круге существует подряд n=12 пикселей, которые темнее, или 12 пикселей, которые светлее, чем центр.

Как показала практика, в среднем для принятия решения нужно было проверить около 9 точек. Для того, чтобы ускорить процесс, авторы предложили сначала проверить только четыре пиксела под номерами: 1, 5, 9, 13. Если среди них есть 3 пиксела светлее или темнее, то выполняется полная проверка по 16 точкам, иначе – точка сразу помечается как «не особая». Это сильно сокращает время работы, для принятия решения в среднем достаточно опросить всего лишь около 4 точек окружности.

---

#### BRIEF

![](./assets/lesson_2_image_6.jpg)

Алгоритм вычисления дескрипторов особых точек. строится из 256 бинарных сравнений между яркостями пикселей на размытом изображении. Бинарный тест между точками х и у определяется так:

$$
\tau(P, x, y) := 
\begin{cases}
1 & \text{if } p(x) < p(y), \\
0 & \text{if } p(x) \geq p(y).
\end{cases}
$$

В оригинальной статье было рассмотрено несколько способов выбора точек для бинарных сравнений. Как оказалось, один из лучших способов – выбирать точки случайным образом Гауссовским распределением вокруг центрального пиксела. Эта случайная последовательность точек выбирается один раз и в дальнейшем не меняется. Размер рассматриваемой окрестности точки равен 31х31 пиксел, а апертура размытия равна 5.

Полученный бинарный дескриптор оказывается устойчив к сменам освещения, перспективному искажению, быстро вычисляется и сравнивается, но очень неустойчив к вращению в плоскости изображения.

### ORB 

(Oriented FAST and Rotated BRIEF)

1. Детектор ключевых точек: FAST
2. Оценка ключевых точек
- считает Harris score для точки
- оставляет топ-N точек
3. Устойчивость к масштабу: пирамида изображений
- строится пирамида: изображение в нескольких масштабах (уменьшенные копии)
- FAST запускается на каждом уровне
4. Ориентация ключевой точки (Oriented)
- смотрим на распределение яркости вокруг точки
- находим куда смещён центр массы яркости
1. Дескриптор: Rotated BRIEF
- ORB делает rBRIEF: набор пар точек для сравнений поворачивается согласно ориентации ключевой точки

In [ ]:
img = cv2.imread(OUTDOOR_FIRST_PART_PATH, cv2.IMREAD_GRAYSCALE)

orb = cv2.ORB_create()
kp = orb.detect(img,None)
kp, des = orb.compute(img, kp)
 
img2 = cv2.drawKeypoints(cv2.imread(OUTDOOR_FIRST_PART_PATH), kp, None, color=(0,0,255), flags=0)
show_image(img2[..., ::-1])

---

## Matching

### Brute Force Matcher

#### Ratio Test (Lowe)

Соответствия ключевых точек между двумя изображениями находятся путём определения их ближайших соседей. Однако в некоторых случаях второе ближайшее совпадение может быть очень близко к первому. Это может происходить из-за шума или по другим причинам. В таком случае вычисляется отношение расстояния до ближайшего соседа к расстоянию до второго ближайшего. Если это отношение больше 0,8, совпадение отбрасывается. Согласно статье, такой подход устраняет около 90% ложных совпадений, теряя при этом лишь 5% правильных. ([source](https://docs.opencv.org/4.x/da/df5/tutorial_py_sift_intro.html))

In [ ]:
img1 = cv2.imread(OUTDOOR_FIRST_PART_PATH, cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(OUTDOOR_SECOND_PART_PATH, cv2.IMREAD_GRAYSCALE)

sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# Матчинг дескрипторов
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
matches_knn = bf.knnMatch(des1, des2, k=2)

good = []
ratio = 0.8
for m, n in matches_knn:
    if m.distance < ratio * n.distance:
        good.append(m)


good = sorted(good, key=lambda x: x.distance)
out = cv2.drawMatches(
    cv2.imread(OUTDOOR_FIRST_PART_PATH), kp1,
    cv2.imread(OUTDOOR_SECOND_PART_PATH), kp2,
    good[:100], None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

show_image(out[..., ::-1])

### FLANN (Fast Library for Approximate Nearest Neighbors)

In [ ]:
img1 = cv2.imread(OUTDOOR_FIRST_PART_PATH, cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(OUTDOOR_SECOND_PART_PATH, cv2.IMREAD_GRAYSCALE)

sift = cv2.ORB_create()
kp1 = orb.detect(img1,None)
kp2 = orb.detect(img2,None)

kp1, des1 = sift.compute(img1, kp1)
kp2, des2 = sift.compute(img2, kp2)

# Матчинг дескрипторов
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)

matches = sorted(matches, key=lambda m: m.distance)

top_n = 50
matched_vis = cv2.drawMatches(
    cv2.imread(OUTDOOR_FIRST_PART_PATH), kp1,
    cv2.imread(OUTDOOR_SECOND_PART_PATH), kp2,
    matches[:top_n], None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

show_image(matched_vis[..., ::-1])

---

### FLANN

In [ ]:
img1 = cv2.imread(OUTDOOR_FIRST_PART_PATH, cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(OUTDOOR_SECOND_PART_PATH, cv2.IMREAD_GRAYSCALE)

sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)
matches_knn = flann.knnMatch(des1, des2, k=2)

good = []
ratio = 0.75
for m, n in matches_knn:
    if m.distance < ratio * n.distance:
        good.append(m)

out = cv2.drawMatches(
    cv2.imread(OUTDOOR_FIRST_PART_PATH), kp1,
    cv2.imread(OUTDOOR_SECOND_PART_PATH), kp2,
    good, None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

show_image(out[..., ::-1])

### RANSAC

Random Sample Consensus

Алгоритм:

- случайно выбрать минимальное число точек
- построить модель
- проверить, сколько точек согласуются с моделью
- повторить много раз
- выбрать модель с наибольшим числом согласованных точек

In [ ]:
# ==============================================================
# ============== код аналогичен клетке выше ====================
# ==============================================================
img1 = cv2.imread(OUTDOOR_FIRST_PART_PATH, cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(OUTDOOR_SECOND_PART_PATH, cv2.IMREAD_GRAYSCALE)

sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)
matches_knn = flann.knnMatch(des1, des2, k=2)

good = []
ratio = 0.75
for m, n in matches_knn:
    if m.distance < ratio * n.distance:
        good.append(m)
# ==============================================================

# RANSAC
src_pts = np.float32([kp1[m.queryIdx].pt for m in good])
dst_pts = np.float32([kp2[m.trainIdx].pt for m in good])

H, mask = cv2.findHomography(
    src_pts,
    dst_pts,
    cv2.RANSAC,
    5.0
)
inliers = [good[i] for i in range(len(good)) if mask[i]]

print("Matches after ratio test:", len(good))
print("Inliers after RANSAC:", len(inliers))

img_matches = cv2.drawMatches(
    cv2.imread(OUTDOOR_FIRST_PART_PATH), kp1,
    cv2.imread(OUTDOOR_SECOND_PART_PATH), kp2,
    inliers,
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

show_image(img_matches[..., ::-1])

In [ ]:
# Посмотрим на найденную ransac гомографию

fig, ax = plt.subplots(1, 2, figsize=(8, 6))

ax[0].imshow(cv2.warpPerspective(img1, H, (img1.shape[1], img2.shape[0])), cmap="gray")
ax[0].axis('off')
ax[1].imshow(img2, cmap="gray")
ax[1].axis('off')
plt.show()

## Классические детекторы

### Viola-Jones (Haar Cascade)

[Подробнее](https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html)


**Идея**

Это классический детектор объектов (часто — лиц), который:

1. Сканирует изображение "окном" разных размеров (scale pyramid)
2. В каждом окне быстро считает простые признаки (Haar-признаки)
3. Прогоняет окно через каскад простых классификаторов и рано отбрасывает почти все окна как False
4. Для оставшихся окон делает NMS пересекающихся детекций.

**Haar-like features**

![](assets/lesson_2_image_7.jpg)

**Каскад и обучение**

Каждый этап каскада - это сильный классификатор *AdaBoost*, собранный из слабых:

- Слабый классификатор: если конкретный Haar-признак > порога, это лицо, иначе нет
- AdaBoost выбирает наиболее полезные признаки и веса.


Пример: [haar-cascade.py](./haar-cascade.py)

### HOG + SVM

[Подробнее про HOG](https://learnopencv.com/histogram-of-oriented-gradients/)

**Интуиция**

- Вместо яркости — использовать градиенты (объект определяется формой)

**Алгоритм**

- считаем градиенты изображения
- строим гистограммы направлений
- нормализуем блоки
- классифицируем окно через SVM

Пример: [hog.py](hog.py)

## Camera Calibration

Из-за несовершенства камер и их линз мы можем видеть всякого рода [distortions](https://en.wikipedia.org/wiki/Distortion_%28optics%29)

![](./assets/lesson_2_image_8.jpg)

Два основных вида искажений – radial & tangential

*Radial distortion*
$$
\begin{matrix}
x_{distorted} = x( 1 + k_1 r^2 + k_2 r^4 + k_3 r^6) \\
y_{distorted} = y( 1 + k_1 r^2 + k_2 r^4 + k_3 r^6)
\end{matrix}
$$

*tangential distortion*
$$
\begin{matrix}
x_{distorted} = x + [ 2p_1xy + p_2(r^2+2x^2)] \\
y_{distorted} = y + [ p_1(r^2+ 2y^2)+ 2p_2xy]
\end{matrix}
$$

Для их исправления искажений необходимо найти

$$
Distortion \; coefficients=(k_1 \hspace{10pt} k_2 \hspace{10pt} p_1 \hspace{10pt} p_2 \hspace{10pt} k_3)
$$

В дополнение к этому, необходимо найти *Intrinsic (внутренние)* и *extrinsic (внешние)* параметры камеры

*Intrinsic* параметры включают в себя *focal length (фокусное расстояние)* и *optical centers (оптический центр)*. Фокусное расстояние и оптические центры могут быть использованы для построения матрицы камеры, которая применяется для устранения искажений, вызванных линзами конкретной камеры. Матрица камеры является уникальной для данной камеры, поэтому после вычисления её можно повторно использовать для других изображений, снятых той же камерой. Она выражается в виде матрицы размером 3×3.

$$
camera \; matrix = \left [ \begin{matrix}   f_x & 0 & c_x \\  0 & f_y & c_y \\   0 & 0 & 1 \end{matrix} \right ]
$$

Внешние параметры (extrinsic parameters) соответствуют векторам вращения и переноса, которые переводят координаты 3D-точки в систему координат камеры.

$$
x\approx K[R∣t]X
$$

где $x$ - пиксель, $X$ - координаты точки 3d, $K – camera \; matrix$, $R$ - матрица поворота, $t$ - вектор сдвига

In [ ]:
# Подготовка, скачивание файлов
os.makedirs("chessboard", exist_ok=True)

for i in range(1, 15):
    if os.path.exists(f"chessboard/left{str(i).zfill(2)}.jpg"):
        continue
    # there is no left10.jpg image
    if i == 10:
        continue
    url = f"https://raw.githubusercontent.com/opencv/opencv/refs/heads/master/samples/data/left{str(i).zfill(2)}.jpg"
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        response = requests.get(url, verify=False)
    with open(f"chessboard/left{str(i).zfill(2)}.jpg", "wb") as f:
        f.write(response.content)

In [ ]:
img_paths = sorted(glob.glob("chessboard/*.jpg"))
assert len(img_paths) > 0, "Не нашел изображений в chessboard/*.jpg"

show_image(cv2.imread(img_paths[10])[..., ::-1])

In [ ]:
pattern_size = (9, 6)

objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-3)

objpoints = []
imgpoints = []
good = []

img_shape = None

In [ ]:
# Ищем подходящие кадры и координаты углов клеток
for p in img_paths:
    img = cv2.imread(p)
    if img is None:
        continue

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_shape = gray.shape[::-1]

    found, corners = cv2.findChessboardCorners(
        gray, pattern_size,
        flags=cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
    )

    if not found:
        print("Не найдено углов:", Path(p).name)
        continue

    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

    objpoints.append(objp)
    imgpoints.append(corners2)
    good.append(p)

print("Подходящих изображений:", len(good))
assert len(good) >= 5, "Слишком мало удачных кадров — попробуй больше изображений или проверь pattern_size"

In [ ]:
sample_path = good[0]
img = cv2.imread(sample_path)
vis = img.copy()
plt.title(f"Углы: {Path(sample_path).name}")
show_image(cv2.drawChessboardCorners(vis, pattern_size, imgpoints[0], True))

In [ ]:
# достаем параметры камеры
ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_shape, None, None)

print("RMS (ret):", ret)
print("K:\n", K)
print("dist:", dist.ravel())
print("Views:", len(rvecs))

In [ ]:
test_path = good[0]
img = cv2.imread(test_path)
h, w = img.shape[:2]

# https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html
# If the scaling parameter alpha=0, it returns undistorted image with minimum unwanted pixels.
# So it may even remove some pixels at image corners.
# If alpha=1, all pixels are retained with some extra black images
newK, roi = cv2.getOptimalNewCameraMatrix(K, dist, (w, h), alpha=1.0)
undist = cv2.undistort(img, K, dist, None, newK)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title("Оригинал")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(undist, cv2.COLOR_BGR2RGB))
plt.title("cv2.undistort")
plt.axis("off")
plt.show()

---

### PnP

**Perspective-n-Point**


*Аналогия*
- на столе лежит шахматка
- ты знаешь координаты её углов
- кто-то сделал фотографию
- тебе дали только фото

Вопрос: откуда была сделана фотография?

То есть нужно найти:

- позицию камеры
- ориентацию камеры

так, чтобы проекция 3D-точек совпала с наблюдаемыми пикселями.


*Алгоритм*
- Берёт предположение о положении камеры
- Проецирует 3D точки в изображение
- Сравнивает с реальными пикселями
- Двигает камеру так, чтобы ошибка уменьшалась

То есть минимизируется:

$$
\sum_i ||x_i - \hat{x}_i(R,t)||^2
$$

где:

- $x_i$ – наблюдаемая точка
- $\hat{x}_i$ – спроецированная точка

In [ ]:
indx = 3
test_path = good[indx]
img = cv2.imread(test_path)

ok, rvec, tvec = cv2.solvePnP(objp, imgpoints[indx], K, dist)

vis = img.copy()
cv2.drawFrameAxes(vis, K, dist, rvec, tvec, length=3)

plt.title("solvePnP")
show_image(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))

---

### Kalman Filter

[Википедия](https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D0%BB%D1%8C%D1%82%D1%80_%D0%9A%D0%B0%D0%BB%D0%BC%D0%B0%D0%BD%D0%B0)

![](./assets/lesson_2_image_9.png)

Разберем только 1D пример. В размерностях больше принцип тот же, но математических выкладок больше

Фильтр Калмана — это алгоритм оценки истинного состояния системы по шумным измерениям, использующий:

- модель динамики системы,
- статистические свойства ошибок.

**Постановка задачи**

* $x_k$ - истинное состояние системы
* $z_k$ - измерение
* $u_k$ - управляющее воздействие
* $\psi_k$ - ошибка модели
* $\eta_k$ - ошибка измерения

$$
x_{k+1} = f(x_k, u_k) + \psi_k
$$

$$
z_k = x_k + \eta_k
$$

Предположения:

1. Ошибки имеют нулевое среднее:

$$
\mathbb{E}[\psi_k] = 0, \qquad \mathbb{E}[\eta_k] = 0
$$

2. Известны их дисперсии:

$$
\mathrm{Var}(\psi_k), \quad \mathrm{Var}(\eta_k)
$$

3. Ошибки независимы между собой и по времени


**Алгоритм**

Есть:

- предсказание модели $\hat{x}_{k|k-1}$
- измерение $z_k$

Оценка строится как линейная комбинация:

$$
\hat{x}_k =
(1-K_k)\hat{x}_{k|k-1} + K_k z_k
$$

$K_k$ — **коэффициент Калмана**.

**Вывод коэффициента Калмана**

Ошибка оценки:

$$
e_k = \hat{x}_k - x_k
$$

Хотим минимизировать квадрат ошибки

$$
\mathbb{E}[e_k^2] \rightarrow \min
$$

Минимизация приводит к:

$$
K_k =
\frac{P_{k|k-1}}
{P_{k|k-1} + R}
$$

где:

- $P_{k|k-1}$ дисперсия ошибки предсказания
- $R$ дисперсия ошибки измерения


После коррекции:

$$
P_k = (1 - K_k) P_{k|k-1}
$$


#### Предсказание

$$
\hat{x}_{k|k-1} = f(\hat{x}_{k-1}, u_k)
$$

$$
P_{k|k-1} = P_{k-1} + Q
$$

где $Q$ дисперсия ошибки модели.


#### Коррекция

$$
K_k = \frac{P_{k|k-1}}{P_{k|k-1} + R}
$$


$$
\hat{x}_k= \hat{x}_{k|k-1}
 + K_k(z_k - \hat{x}_{k|k-1})$$


$$
P_k = (1-K_k)P_{k|k-1}
$$


Пример: [kalman.py](./kalman.py)

### Panorama stitching

In [ ]:
DEBUG_PLOTS = True

In [ ]:
# helpers
def nonzero_mask(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask = (gray > 0).astype(np.uint8) * 255
    return mask

def crop_to_content(img):
    mask = nonzero_mask(img)
    ys, xs = np.where(mask > 0)
    if len(xs) == 0 or len(ys) == 0:
        return img
    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    return img[y0:y1+1, x0:x1+1]

# detection && description
def create_feature(name):
    if name == "SIFT":
        return cv2.SIFT_create(nfeatures=4000)
    else:
        return cv2.ORB_create(nfeatures=6000)

def detect_and_describe(feature, img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kps, desc = feature.detectAndCompute(gray, None)
    return kps, desc

# FLANN
def create_flann(kind):
    if kind == "SIFT":
        index_params = dict(algorithm=1, trees=5)   # 1 = FLANN_INDEX_KDTREE
        search_params = dict(checks=50)
    else:
        index_params = dict(
            algorithm=6,      # 6 = FLANN_INDEX_LSH
            table_number=12,
            key_size=20,
            multi_probe_level=2
        )
        search_params = dict(checks=50)

    return cv2.FlannBasedMatcher(index_params, search_params)

# RANSAC
def homography_ransac(kpsA, kpsB, matches, reproj_thresh=4.0):
    if len(matches) < 4:
        return None, None

    ptsA = np.float32([kpsA[m.queryIdx].pt for m in matches])
    ptsB = np.float32([kpsB[m.trainIdx].pt for m in matches])

    H, mask = cv2.findHomography(ptsB, ptsA, cv2.RANSAC, reproj_thresh)
    return H, mask  # mask: 1=inlier, 0=outlier

def match_flann(flann, kind, descA, descB, ratio=0.75):
    """
    KNN matching + ratio test (Lowe).
    """
    if descA is None or descB is None:
        return []

    if kind == "SIFT":
        descA = descA.astype(np.float32)
        descB = descB.astype(np.float32)

    knn = flann.knnMatch(descA, descB, k=2)

    good = []
    for m, n in knn:
        if m.distance < ratio * n.distance:
            good.append(m)
    return good

# составление композиции 2х изображений
def estimate_canvas(imgA, imgB, H):
    """
    Оцениваем размер холста для imgA + warped(imgB)
    Возвращаем (canvas_w, canvas_h), offset=(ox, oy)
    """
    hA, wA = imgA.shape[:2]
    hB, wB = imgB.shape[:2]

    cornersB = np.float32([[0, 0], [wB, 0], [wB, hB], [0, hB]]).reshape(-1, 1, 2)
    warpedCornersB = cv2.perspectiveTransform(cornersB, H)

    cornersA = np.float32([[0, 0], [wA, 0], [wA, hA], [0, hA]]).reshape(-1, 1, 2)

    all_corners = np.vstack((cornersA, warpedCornersB)).reshape(-1, 2)

    min_xy = all_corners.min(axis=0)
    max_xy = all_corners.max(axis=0)

    min_x, min_y = min_xy
    max_x, max_y = max_xy

    ox = int(np.floor(-min_x)) if min_x < 0 else 0
    oy = int(np.floor(-min_y)) if min_y < 0 else 0

    canvas_w = int(np.ceil(max_x - min_x))
    canvas_h = int(np.ceil(max_y - min_y))

    return (canvas_w, canvas_h), (ox, oy)


def warp_to_canvas(img, H, canvas_size, offset):
    """Варпим img на общий холст, добавляя сдвиг offset"""
    ox, oy = offset
    T = np.array([[1, 0, ox],
                  [0, 1, oy],
                  [0, 0, 1]], dtype=np.float64)
    Ht = T @ H
    warped = cv2.warpPerspective(img, Ht, canvas_size)
    return warped


def compose_canvas(img_ref, canvas, offset):
    """
    Кладём reference-изображение (img_ref) на уже созданный canvas
    """
    ox, oy = offset
    h, w = img_ref.shape[:2]
    out = canvas.copy()
    out[oy:oy+h, ox:ox+w] = img_ref
    return out

def simple_overlay(base, overlay):
    """
    Простейшее смешивание
    - Берём все непустые пиксели overlay и кладём их на base
    """
    out = base.copy()
    m = nonzero_mask(overlay)
    out[m > 0] = overlay[m > 0]
    return out

# функции для отрисовки
def draw_matches(imgA, kpsA, imgB, kpsB, matches, inlier_mask=None, title="matches"):
    """
    Рисуем matches
    - Если inlier_mask задан, рисуем только inliers.
    """
    if not DEBUG_PLOTS:
        return

    if inlier_mask is not None:
        inlier_mask = inlier_mask.ravel().astype(bool)
        matches = [m for m, ok in zip(matches, inlier_mask) if ok]

    vis = cv2.drawMatches(
        imgA, kpsA,
        imgB, kpsB,
        matches,
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )
    plt.title(title)
    show_image(vis[..., ::-1])

In [ ]:
def stitch_two_simple_overlay(imgA, imgB, feature, kind):
    """
    Склеивает imgB к imgA:
    1) keypoints/desc
    2) match FLANN
    3) RANSAC homography (B -> A)
    4) warp B на холст + кладём A
    5) blending
    """
    kpsA, descA = detect_and_describe(feature, imgA)
    kpsB, descB = detect_and_describe(feature, imgB)

    if DEBUG_PLOTS:
        fig, ax = plt.subplots(1, 2)
        ax[0].imshow(cv2.drawKeypoints(imgA, kpsA, None)[..., ::-1])
        ax[1].imshow(cv2.drawKeypoints(imgB, kpsB, None)[..., ::-1])
        ax[0].axis('off')
        ax[1].axis('off')
        plt.title(f"Дескрипторы ключевых точек с помощью {kind}")
        plt.show()

    flann = create_flann(kind)
    matches = match_flann(flann, kind, descA, descB, ratio=0.75)

    # matches до RANSAC
    draw_matches(imgA, kpsA, imgB, kpsB, matches, None, "Matches (ratio test)")

    H, inlier_mask = homography_ransac(kpsA, kpsB, matches, reproj_thresh=4.0)
    if H is None or inlier_mask is None:
        raise RuntimeError("Not enough good matches / homography failed.")

    # matches после RANSAC (inliers)
    draw_matches(imgA, kpsA, imgB, kpsB, matches, inlier_mask, "Matches (RANSAC inliers)")

    canvas_size, offset = estimate_canvas(imgA, imgB, H)

    warpedB = warp_to_canvas(imgB, H, canvas_size, offset)
    if DEBUG_PLOTS:
        plt.title("Warped B on canvas")
        show_image(warpedB[..., ::-1])

    base = compose_canvas(imgA, warpedB, offset)
    if DEBUG_PLOTS:
        plt.title("Base (A placed onto canvas)")
        show_image(base[..., ::-1])

    blended = simple_overlay(base, warpedB)
    if DEBUG_PLOTS:
        plt.title("Blended")
        show_image(blended[..., ::-1])

    return blended


In [ ]:
img1 = cv2.imread(OUTDOOR_FIRST_PART_PATH)
img2 = cv2.imread(OUTDOOR_SECOND_PART_PATH)
img3 = cv2.imread(OUTDOOR_THIRD_PART_PATH)

kind = "SIFT"
feature = create_feature(kind)

fig, ax = plt.subplots(1, 3)

ax[0].imshow(img1[..., ::-1])
ax[0].axis("off")
ax[1].imshow(img2[..., ::-1])
ax[1].axis("off")
ax[2].imshow(img3[..., ::-1])
ax[2].axis("off")
plt.show()

In [ ]:
pano12 = stitch_two_simple_overlay(img1, img2, feature, kind)
pano12 = crop_to_content(pano12)

In [ ]:
pano123 = stitch_two_simple_overlay(pano12, img3, feature, kind)
pano123 = crop_to_content(pano123)
pano123_simple = pano123.copy()

#### Poisson blending

[блогпост](https://learnopencv.com/seamless-cloning-using-opencv-python-cpp/)

[документация](https://docs.opencv.org/4.x/df/da0/group__photo__clone.html)

Интуитивно - это способ вставить одну область изображения в другую так, чтобы граница вставки стала незаметной, даже если освещение или цвет отличаются


**Алгоритм**

1. Дано:
- изображение-источник (что нужно вставить)
- изображение-фон (куда вставляется)
- маска - область, где должна происходить вставка

2. Из источника берётся не цвет, а изменения цвета

- где изображение становится светлее, где темнее
- где есть границы объектов и текстуры
- берется структура изменений а не сами значения цвета.
- сохраняются детали, контуры и текстура

3. Граница фиксируется по фону

- По краю области вставки значения берутся из фонового изображения
- граница точно совпадает с фоном
- не возникает резкого скачка цвета на шве.

4. Внутри области пересчитываются пиксели
- Теперь алгоритм заполняет всю область вставки заново
- старается сохранить изменения яркости как в источнике
- но одновременно плавно подстроиться под значения на границе

Если формально, то мы считаем градиент источника и решаем систему линейных уравнений, соответствующих [дискретному уравнению Пуассона](https://ru.wikipedia.org/wiki/%D0%A3%D1%80%D0%B0%D0%B2%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_%D0%9F%D1%83%D0%B0%D1%81%D1%81%D0%BE%D0%BD%D0%B0)

In [ ]:
def poisson_blend(base, overlay):
    # 1) Строим маску overlay:
    # где overlay НЕ чёрный (там есть реальные пиксели после warp)
    # там mask=255, иначе 0
    mask = nonzero_mask(overlay)

    # 2) Если overlay пустой (вдруг он целиком чёрный) — нечего смешивать.
    ys, xs = np.where(mask > 0)
    if len(xs) == 0 or len(ys) == 0:
        return base

    # 3) Находим bounding box области, которую будем "вшивать"
    # Это нужно, чтобы определить центр вставки
    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()

    # 4) Центр вставки. seamlessClone просит "куда вставлять" в координатах base
    # Мы берём центр bbox маски — обычно этого достаточно
    center = (int((x0 + x1) / 2), int((y0 + y1) / 2))

    # seamlessClone(src, dst, mask, center, mode)
    #
    # src = overlay (то, что вставляем)
    # dst = base (фон)
    # mask = область src, которую вставляем
    # center = точка в dst, вокруг которой будет размещён src
    # cv2.NORMAL_CLONE - классический Poisson blending
    blended = cv2.seamlessClone(
        overlay, base, mask, center, cv2.NORMAL_CLONE,
    )

    return blended


In [ ]:
def stitch_two_poisson(imgA, imgB, feature, kind):
    """
    Склеивает imgB к imgA:
    1) keypoints/desc
    2) match FLANN
    3) RANSAC homography (B -> A)
    4) warp B на холст + кладём A
    5) blending
    """
    kpsA, descA = detect_and_describe(feature, imgA)
    kpsB, descB = detect_and_describe(feature, imgB)

    if DEBUG_PLOTS:
        fig, ax = plt.subplots(1, 2)
        ax[0].imshow(cv2.drawKeypoints(imgA, kpsA, None)[..., ::-1])
        ax[1].imshow(cv2.drawKeypoints(imgB, kpsB, None)[..., ::-1])
        ax[0].axis('off')
        ax[1].axis('off')
        plt.title(f"Дескрипторы ключевых точек с помощью {kind}")
        plt.show()

    flann = create_flann(kind)
    matches = match_flann(flann, kind, descA, descB, ratio=0.75)

    # matches до RANSAC
    draw_matches(imgA, kpsA, imgB, kpsB, matches, None, "Matches (ratio test)")

    H, inlier_mask = homography_ransac(kpsA, kpsB, matches, reproj_thresh=4.0)
    if H is None or inlier_mask is None:
        raise RuntimeError("Not enough good matches / homography failed.")

    # matches после RANSAC (inliers)
    draw_matches(imgA, kpsA, imgB, kpsB, matches, inlier_mask, "Matches (RANSAC inliers)")

    canvas_size, offset = estimate_canvas(imgA, imgB, H)

    warpedB = warp_to_canvas(imgB, H, canvas_size, offset)
    if DEBUG_PLOTS:
        plt.title("Warped B on canvas")
        show_image(warpedB[..., ::-1])

    base = compose_canvas(imgA, warpedB, offset)
    if DEBUG_PLOTS:
        plt.title("Base (A placed onto canvas)")
        show_image(base[..., ::-1])

    blended = poisson_blend(base, warpedB)
    if DEBUG_PLOTS:
        plt.title("Blended")
        show_image(blended[..., ::-1])

    return blended


In [ ]:
pano12 = stitch_two_poisson(img1, img2, feature, kind)
pano12 = crop_to_content(pano12)

In [ ]:
pano123 = stitch_two_poisson(pano12, img3, feature, kind)
pano123 = crop_to_content(pano123)
pano123_poisson = pano123.copy()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 6))
ax = ax.flatten()

ax[0].imshow(pano123_simple[..., ::-1])
ax[0].axis('off')
ax[1].imshow(pano123_poisson[..., ::-1])
ax[1].axis('off')
plt.show()

---